# Needle in a Haystack (NIAH) Testing Tutorial

This notebook demonstrates how to use the NIAH Tester to evaluate the Highlights API's ability to find specific text (the needle) within a large collection of distractor texts (the haystack). This type of testing is crucial for assessing the robustness and accuracy of semantic search systems.

## Setup

First, we'll import the required libraries and the HighlightsClient from our previous example.

In [ ]:
import random
from typing import List, Tuple
from examples.api_basics import HighlightsClient

## NIAH Tester Class

Below we define our NIAHTester class that will handle:
1. Generating test data with distractors
2. Running search tests
3. Analyzing and reporting results

In [ ]:
class NIAHTester:
    def __init__(self, api_key: str):
        self.client = HighlightsClient(api_key=api_key)

    def generate_test_data(
        self,
        needle: str,
        num_haystack_items: int = 100
    ) -> Tuple[List[str], int]:
        """
        Generate test data with a needle hidden in a haystack.

        Args:
            needle: The text to find
            num_haystack_items: Number of distractor texts to generate

        Returns:
            Tuple of (list of text chunks, index of needle)
        """
        # Generate distractor texts
        haystack = [
            f"Distractor text number {i}: This is irrelevant content."
            for i in range(num_haystack_items)
        ]

        # Insert needle at random position
        needle_position = random.randint(0, len(haystack))
        haystack.insert(needle_position, needle)

        return haystack, needle_position

    def run_test(
        self,
        needle: str,
        query: str,
        num_haystack_items: int = 100
    ) -> dict:
        """
        Run a needle-in-haystack test.

        Args:
            needle: The text to find
            query: The search query
            num_haystack_items: Number of distractor texts

        Returns:
            Dictionary containing test results
        """
        # Generate test data
        haystack, true_position = self.generate_test_data(
            needle=needle,
            num_haystack_items=num_haystack_items
        )

        # Perform search
        results = self.client.search_text_chunks(
            query=query,
            text_chunks=haystack,
            top_n=1
        )

        # Analyze results
        found_position = None
        if results["results"]:
            found_text = results["results"][0]["text"]
            if found_text == needle:
                found_position = results["results"][0].get("start_idx")

        return {
            "success": found_position == true_position,
            "true_position": true_position,
            "found_position": found_position,
            "total_chunks": len(haystack),
            "metadata": results.get("metadata", {})
        }

## Running a Test

Let's initialize our tester with an API key and run a test case. We'll use a text about transformer architectures as our needle.

In [ ]:
# Initialize the tester
tester = NIAHTester(api_key="your-api-key")

# Define our test case
needle = """
    The transformer architecture revolutionized natural language processing
    by introducing self-attention mechanisms that can process sequential
    data more effectively than previous approaches.
"""

query = "How did transformers change NLP?"

# Run the test
results = tester.run_test(
    needle=needle,
    query=query,
    num_haystack_items=50
)

## Analyzing Results

Now let's examine the results of our test in detail.

In [ ]:
print("NIAH Test Results:")
print(f"Success: {results['success']}")
print(f"True Position: {results['true_position']}")
print(f"Found Position: {results['found_position']}")
print(f"Total Chunks: {results['total_chunks']}")
print("\nMetadata:")
print(results["metadata"])

## Understanding the Results

The test results tell us:
- Whether the API successfully found the needle text (`success`)
- Where the needle was actually placed (`true_position`)
- Where the API found it (`found_position`)
- The total size of the haystack (`total_chunks`)
- Additional metadata from the API response
